In [34]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def scrape_data(state_name: str, district_name: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        async def close_popup():
            try:
                # Wait for popup to appear (multiple possible selectors)
                popup_selectors = [
                    "#onloadModal",
                    ".modal",
                    "[role='dialog']",
                    ".popup",
                    ".modal-dialog"
                ]
                
                popup_found = False
                for selector in popup_selectors:
                    try:
                        await page.wait_for_selector(selector, timeout=2000)
                        popup_found = True
                        print(f"Found popup with selector: {selector}")
                        break
                    except:
                        continue
                
                if popup_found:
                    # Try multiple close button selectors
                    close_selectors = [
                        "#onloadModal button.close",
                        "#onloadModal .btn-close",
                        "#onloadModal .close",
                        ".modal button.close",
                        ".modal .btn-close",
                        ".modal .close",
                        "[aria-label='Close']",
                        "button[data-dismiss='modal']",
                        ".modal-header .close",
                        "button:has-text('×')",
                        "button:has-text('Close')"
                    ]
                    
                    for close_selector in close_selectors:
                        try:
                            close_btn = await page.query_selector(close_selector)
                            if close_btn:
                                await close_btn.click()
                                print(f"Clicked close button: {close_selector}")
                                # Wait a bit for the popup to close
                                await page.wait_for_timeout(1000)
                                return True
                        except Exception as e:
                            continue
                    
                    # If no close button found, try pressing Escape
                    try:
                        await page.keyboard.press("Escape")
                        print("Pressed Escape to close popup")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                        
                    # If still can't close, try clicking outside the modal
                    try:
                        await page.click("body", position={"x": 10, "y": 10})
                        print("Clicked outside modal to close")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                
            except Exception as e:
                print(f"Error in close_popup: {e}")
                pass
            
            return False
        
        async def extract_table_data():
            """Extract data from the current table"""
            try:
                # Wait for results table
                await page.wait_for_selector("table", timeout=15000)
                
                # Extract table headers first
                headers = []
                try:
                    header_cells = await page.query_selector_all("table thead tr th")
                    for header in header_cells:
                        header_text = await header.inner_text()
                        headers.append(header_text.strip())
                except:
                    # Default headers based on the structure
                    headers = [
                        "Serial No", 
                        "Type", 
                        "District", 
                        "Area", 
                        "Dealer name", 
                        "Mobile No", 
                        "Address", 
                        "NaPanta Mobile App"
                    ]
                
                # Extract table rows
                rows = await page.query_selector_all("table tbody tr")
                data = []
                
                for row in rows:
                    cols = await row.query_selector_all("td")
                    row_data = []
                    for col in cols:
                        text = await col.inner_text()
                        row_data.append(text.strip())
                    if row_data:  # Only add non-empty rows
                        data.append(row_data)
                
                return data, headers
            except Exception as e:
                print(f"Error extracting table data: {e}")
                return [], []
        
        all_data = []
        all_headers = []
        
        try:
            # --- Open website ---
            print("Opening website...")
            await page.goto("https://www.napanta.com/seed-dealer", wait_until="domcontentloaded")
            
            # Wait a bit for any popups to load
            await page.wait_for_timeout(2000)
            
            # --- Close popup if appears ---
            print("Checking for initial popup...")
            await close_popup()
            
            # --- Select state ---
            print(f"Selecting state: {state_name}")
            await page.wait_for_selector("#ddlState", timeout=10000)
            await page.select_option("#ddlState", label=state_name)
            
            # Wait for district dropdown to populate
            await page.wait_for_timeout(2000)
            
            # --- Select district ---
            print(f"Selecting district: {district_name}")
            await page.wait_for_selector("#ddlDistrict", timeout=10000)
            await page.select_option("#ddlDistrict", label=district_name)
            
            # Wait for area dropdown to populate
            await page.wait_for_timeout(2000)
            
            # --- Get all area options ---
            print("Getting all available areas...")
            area_dropdown = await page.wait_for_selector("#ddlMarket", timeout=10000)
            area_options = await page.query_selector_all("#ddlMarket option")
            
            # Extract area names (skip the first option which is usually "Select Area" or similar)
            areas = []
            for option in area_options[1:]:  # Skip first option
                area_text = await option.inner_text()
                area_value = await option.get_attribute("value")
                if area_text and area_value and area_text.strip() != "":
                    areas.append(area_text.strip())
            
            print(f"Found {len(areas)} areas: {areas}")
            
            # --- Iterate through each area ---
            for i, area_name in enumerate(areas):
                print(f"\n--- Processing area {i+1}/{len(areas)}: {area_name} ---")
                
                try:
                    # Select the current area
                    await page.select_option("#ddlMarket", label=area_name)
                    await page.wait_for_timeout(1000)
                    
                    # --- Make sure popup is gone before clicking GO ---
                    print("Checking for popup before clicking GO...")
                    await close_popup()
                    
                    # --- Click GO button ---
                    print("Clicking GO button...")
                    
                    # Wait for GO button and ensure it's clickable
                    go_button = await page.wait_for_selector("button.go-btn", timeout=10000)
                    
                    # Scroll to button if needed
                    await go_button.scroll_into_view_if_needed()
                    
                    # Make sure button is enabled and visible
                    is_enabled = await go_button.is_enabled()
                    is_visible = await go_button.is_visible()
                    
                    print(f"GO button - Enabled: {is_enabled}, Visible: {is_visible}")
                    
                    if not is_enabled or not is_visible:
                        print("GO button not ready, waiting...")
                        await page.wait_for_timeout(2000)
                    
                    await go_button.click()
                    print("GO button clicked successfully")
                    
                    # --- Close popup again if it reappears after reload ---
                    await page.wait_for_timeout(2000)
                    print("Checking for popup after GO click...")
                    await close_popup()
                    
                    # --- Extract table data for current area ---
                    print(f"Extracting table data for area: {area_name}")
                    area_data, headers = await extract_table_data()
                    
                    if area_data:
                        all_data.extend(area_data)
                        if not all_headers and headers:
                            all_headers = headers
                        print(f"Extracted {len(area_data)} rows for {area_name}")
                    else:
                        print(f"No data found for area: {area_name}")
                    
                    # Wait a bit before processing next area
                    await page.wait_for_timeout(1000)
                    
                except Exception as e:
                    print(f"Error processing area '{area_name}': {e}")
                    continue
            
            print(f"\n--- Completed processing all areas ---")
            print(f"Total rows extracted: {len(all_data)}")
            
        except Exception as e:
            print(f"Error during scraping: {e}")
            # Take a screenshot for debugging
            await page.screenshot(path="error_screenshot.png")
            raise e
        finally:
            await browser.close()
        
        # Create consolidated DataFrame
        if all_data:
            # Use extracted headers if available, otherwise use default column names
            if not all_headers:
                all_headers = [
                    "Serial No", 
                    "Type", 
                    "District", 
                    "Area", 
                    "Dealer name", 
                    "Mobile No", 
                    "Address", 
                    "NaPanta Mobile App"
                ]
            
            # Ensure we have the right number of columns
            if all_data and len(all_data[0]) != len(all_headers):
                print(f"Warning: Data has {len(all_data[0])} columns but {len(all_headers)} headers")
                # Adjust headers to match data length
                if len(all_data[0]) > len(all_headers):
                    for i in range(len(all_headers), len(all_data[0])):
                        all_headers.append(f"Column_{i+1}")
                else:
                    all_headers = all_headers[:len(all_data[0])]
            
            df = pd.DataFrame(all_data, columns=all_headers)
            
            # Clean up the data
            for col in df.columns:
                df[col] = df[col].astype(str).str.strip()
                # Replace empty strings and dashes with None
                df[col] = df[col].replace(['', '-', 'nan'], None)
            
            # Reset index to have sequential numbering
            df.reset_index(drop=True, inplace=True)
            
            print(f"DataFrame created with columns: {list(df.columns)}")
            return df
        else:
            print("No data extracted from any area")
            return pd.DataFrame()

# -----------------------------
# Run the scraper
# -----------------------------
if __name__ == "__main__":
    state_name = "Uttar Pradesh"
    district_name = "Lucknow"
    
    try:
        df = asyncio.run(scrape_data(state_name, district_name))
        print("\nConsolidated data from all areas:")
        print(df.head(10))  # Show first 10 rows
        print(f"\nTotal rows: {len(df)}")
        print(f"Columns: {list(df.columns)}")
        
        # Show area breakdown
        if not df.empty and 'Area' in df.columns:
            print(f"\nArea breakdown:")
            area_counts = df['Area'].value_counts()
            for area, count in area_counts.items():
                print(f"  {area}: {count} dealers")
                
        # Save to CSV for easy access
        if not df.empty:
            filename = f"seed_dealers_{state_name}_{district_name}.csv"
            df.to_csv(filename, index=False)
            print(f"\nData saved to: {filename}")
            
    except Exception as e:
        print(f"Scraping failed: {e}")

Opening website...
Checking for initial popup...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Selecting state: Uttar Pradesh
Selecting district: Lucknow
Getting all available areas...
Found 8 areas: ['Bakshi-Ka-Talab', 'Chinhat', 'Gosaiganj', 'Kakori', 'Mal', 'Malihabad', 'Mohanlalganj', 'Sarojaninagar']

--- Processing area 1/8: Bakshi-Ka-Talab ---
Checking for popup before clicking GO...


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator(".popup") to be visible\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator(".popup") to be visible

pipe closed by peer or os.write(pipe, data) raised exception.


Clicking GO button...
Error processing area 'Bakshi-Ka-Talab': Page.wait_for_selector: Target page, context or browser has been closed

--- Processing area 2/8: Chinhat ---
Error processing area 'Chinhat': Page.select_option: Target page, context or browser has been closed

--- Processing area 3/8: Gosaiganj ---
Error processing area 'Gosaiganj': Page.select_option: Connection closed while reading from the driver

--- Processing area 4/8: Kakori ---
Error processing area 'Kakori': Page.select_option: Connection closed while reading from the driver

--- Processing area 5/8: Mal ---
Error processing area 'Mal': Page.select_option: Connection closed while reading from the driver

--- Processing area 6/8: Malihabad ---
Error processing area 'Malihabad': Page.select_option: Connection closed while reading from the driver

--- Processing area 7/8: Mohanlalganj ---
Error processing area 'Mohanlalganj': Page.select_option: Connection closed while reading from the driver

--- Processing area 8/

In [35]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def get_markets(state_name: str, district_name: str):
    """
    Part 1: Get all available markets for a given state and district
    Returns: List of market names
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        async def close_popup():
            try:
                popup_selectors = [
                    "#onloadModal",
                    ".modal",
                    "[role='dialog']",
                    ".popup",
                    ".modal-dialog"
                ]
                
                popup_found = False
                for selector in popup_selectors:
                    try:
                        await page.wait_for_selector(selector, timeout=2000)
                        popup_found = True
                        print(f"Found popup with selector: {selector}")
                        break
                    except:
                        continue
                
                if popup_found:
                    close_selectors = [
                        "#onloadModal button.close",
                        "#onloadModal .btn-close",
                        "#onloadModal .close",
                        ".modal button.close",
                        ".modal .btn-close",
                        ".modal .close",
                        "[aria-label='Close']",
                        "button[data-dismiss='modal']",
                        ".modal-header .close",
                        "button:has-text('×')",
                        "button:has-text('Close')"
                    ]
                    
                    for close_selector in close_selectors:
                        try:
                            close_btn = await page.query_selector(close_selector)
                            if close_btn:
                                await close_btn.click()
                                print(f"Clicked close button: {close_selector}")
                                await page.wait_for_timeout(1000)
                                return True
                        except:
                            continue
                    
                    # Try pressing Escape
                    try:
                        await page.keyboard.press("Escape")
                        print("Pressed Escape to close popup")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                        
                    # Try clicking outside the modal
                    try:
                        await page.click("body", position={"x": 10, "y": 10})
                        print("Clicked outside modal to close")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                
            except Exception as e:
                print(f"Error in close_popup: {e}")
                pass
            
            return False
        
        try:
            # --- Open website ---
            print("Opening website...")
            await page.goto("https://www.napanta.com/seed-dealer", wait_until="domcontentloaded")
            await page.wait_for_timeout(2000)
            
            # --- Close initial popup ---
            print("Checking for initial popup...")
            await close_popup()
            
            # --- Select state ---
            print(f"Selecting state: {state_name}")
            await page.wait_for_selector("#ddlState", timeout=10000)
            await page.select_option("#ddlState", label=state_name)
            await page.wait_for_timeout(2000)
            await close_popup()
            
            # --- Select district ---
            print(f"Selecting district: {district_name}")
            await page.wait_for_selector("#ddlDistrict", timeout=10000)
            await page.select_option("#ddlDistrict", label=district_name)
            await page.wait_for_timeout(2000)
            await close_popup()
            
            # --- Get all market options ---
            print("Getting all available markets...")
            await page.wait_for_selector("#ddlMarket", timeout=10000)
            market_options = await page.query_selector_all("#ddlMarket option")
            
            # Extract market names (skip the first option which is usually "Select Market" or similar)
            markets = []
            for option in market_options[1:]:  # Skip first option
                market_text = await option.inner_text()
                market_value = await option.get_attribute("value")
                if market_text and market_value and market_text.strip() != "":
                    markets.append(market_text.strip())
            
            print(f"Found {len(markets)} markets: {markets}")
            return markets
            
        except Exception as e:
            print(f"Error getting markets: {e}")
            await page.screenshot(path="error_screenshot_markets.png")
            return []
        finally:
            await browser.close()


async def scrape_market_data(state_name: str, district_name: str, market_name: str):
    """
    Part 2: Get data for a specific state, district, and market combination
    Returns: DataFrame with seed dealer data for that market
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        async def close_popup():
            try:
                popup_selectors = [
                    "#onloadModal",
                    ".modal",
                    "[role='dialog']",
                    ".popup",
                    ".modal-dialog"
                ]
                
                popup_found = False
                for selector in popup_selectors:
                    try:
                        await page.wait_for_selector(selector, timeout=2000)
                        popup_found = True
                        print(f"Found popup with selector: {selector}")
                        break
                    except:
                        continue
                
                if popup_found:
                    close_selectors = [
                        "#onloadModal button.close",
                        "#onloadModal .btn-close",
                        "#onloadModal .close",
                        ".modal button.close",
                        ".modal .btn-close",
                        ".modal .close",
                        "[aria-label='Close']",
                        "button[data-dismiss='modal']",
                        ".modal-header .close",
                        "button:has-text('×')",
                        "button:has-text('Close')"
                    ]
                    
                    for close_selector in close_selectors:
                        try:
                            close_btn = await page.query_selector(close_selector)
                            if close_btn:
                                await close_btn.click()
                                print(f"Clicked close button: {close_selector}")
                                await page.wait_for_timeout(1000)
                                return True
                        except:
                            continue
                    
                    # Try pressing Escape
                    try:
                        await page.keyboard.press("Escape")
                        print("Pressed Escape to close popup")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                        
                    # Try clicking outside the modal
                    try:
                        await page.click("body", position={"x": 10, "y": 10})
                        print("Clicked outside modal to close")
                        await page.wait_for_timeout(1000)
                        return True
                    except:
                        pass
                
            except Exception as e:
                print(f"Error in close_popup: {e}")
                pass
            
            return False
        
        async def extract_table_data():
            """Extract data from the current table"""
            try:
                await page.wait_for_selector("table", timeout=15000)
                
                # Extract table headers first
                headers = []
                try:
                    header_cells = await page.query_selector_all("table thead tr th")
                    for header in header_cells:
                        header_text = await header.inner_text()
                        headers.append(header_text.strip())
                except:
                    # Default headers based on the structure
                    headers = [
                        "Serial No", 
                        "Type", 
                        "District", 
                        "Area", 
                        "Dealer name", 
                        "Mobile No", 
                        "Address", 
                        "NaPanta Mobile App"
                    ]
                
                # Extract table rows
                rows = await page.query_selector_all("table tbody tr")
                data = []
                
                for row in rows:
                    cols = await row.query_selector_all("td")
                    row_data = []
                    for col in cols:
                        text = await col.inner_text()
                        row_data.append(text.strip())
                    if row_data and len(row_data) > 1:  # Only add non-empty rows
                        data.append(row_data)
                
                return data, headers
            except Exception as e:
                print(f"Error extracting table data: {e}")
                return [], []
        
        try:
            # --- Open website ---
            print("Opening website...")
            await page.goto("https://www.napanta.com/seed-dealer", wait_until="domcontentloaded")
            await page.wait_for_timeout(2000)
            
            # --- Close initial popup ---
            print("Checking for initial popup...")
            await close_popup()
            
            # --- Select state ---
            print(f"Selecting state: {state_name}")
            await page.wait_for_selector("#ddlState", timeout=10000)
            await page.select_option("#ddlState", label=state_name)
            await page.wait_for_timeout(2000)
            await close_popup()
            
            # --- Select district ---
            print(f"Selecting district: {district_name}")
            await page.wait_for_selector("#ddlDistrict", timeout=10000)
            await page.select_option("#ddlDistrict", label=district_name)
            await page.wait_for_timeout(2000)
            await close_popup()
            
            # --- Select market ---
            print(f"Selecting market: {market_name}")
            await page.wait_for_selector("#ddlMarket", timeout=10000)
            await page.select_option("#ddlMarket", label=market_name)
            await page.wait_for_timeout(1000)
            await close_popup()
            
            # --- Click GO button ---
            print("Clicking GO button...")
            go_button = await page.wait_for_selector("button.go-btn", timeout=10000)
            await go_button.scroll_into_view_if_needed()
            
            is_enabled = await go_button.is_enabled()
            is_visible = await go_button.is_visible()
            print(f"GO button - Enabled: {is_enabled}, Visible: {is_visible}")
            
            if not is_enabled or not is_visible:
                print("GO button not ready, waiting...")
                await page.wait_for_timeout(2000)
            
            await go_button.click()
            print("GO button clicked successfully")
            
            # --- Close popup after GO click ---
            await page.wait_for_timeout(3000)
            print("Checking for popup after GO click...")
            await close_popup()
            
            # --- Extract table data ---
            print(f"Extracting table data for market: {market_name}")
            data, headers = await extract_table_data()
            
            if data:
                print(f"Extracted {len(data)} rows for {market_name}")
            else:
                print(f"No data found for {market_name}")
                return pd.DataFrame()
            
        except Exception as e:
            print(f"Error during scraping: {e}")
            await page.screenshot(path="error_screenshot_market_data.png")
            return pd.DataFrame()
        finally:
            await browser.close()
        
        # Create DataFrame
        if data:
            # Use extracted headers if available, otherwise use default column names
            if not headers:
                headers = [
                    "Serial No", 
                    "Type", 
                    "District", 
                    "Area", 
                    "Dealer name", 
                    "Mobile No", 
                    "Address", 
                    "NaPanta Mobile App"
                ]
            
            # Ensure we have the right number of columns
            if data and len(data[0]) != len(headers):
                print(f"Warning: Data has {len(data[0])} columns but {len(headers)} headers")
                # Adjust headers to match data length
                if len(data[0]) > len(headers):
                    for i in range(len(headers), len(data[0])):
                        headers.append(f"Column_{i+1}")
                else:
                    headers = headers[:len(data[0])]
            
            df = pd.DataFrame(data, columns=headers)
            
            # Clean up the data
            for col in df.columns:
                df[col] = df[col].astype(str).str.strip()
                # Replace empty strings and dashes with None
                df[col] = df[col].replace(['', '-', 'nan'], None)
            
            df.reset_index(drop=True, inplace=True)
            print(f"DataFrame created with columns: {list(df.columns)}")
            return df
        else:
            return pd.DataFrame()


# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    state_name = "Uttar Pradesh"
    district_name = "Lucknow"
    
    # Part 1: Get all available markets
    print("=== PART 1: Getting all markets ===")
    try:
        markets = asyncio.run(get_markets(state_name, district_name))
        print(f"\nAvailable markets in {district_name}, {state_name}:")
        for i, market in enumerate(markets, 1):
            print(f"{i}. {market}")
    except Exception as e:
        print(f"Failed to get markets: {e}")
        markets = []
    
    # Part 2: Get data for a specific market (example with first market)
    if markets:
        print(f"\n=== PART 2: Getting data for specific market ===")
        sample_market = markets[0]  # Take first market as example
        print(f"Getting data for market: {sample_market}")
        
        try:
            df = asyncio.run(scrape_market_data(state_name, district_name, sample_market))
            print(f"\nData for {sample_market}:")
            print(f"Rows: {len(df)}")
            if not df.empty:
                print(f"Columns: {list(df.columns)}")
                print("\nFirst 5 rows:")
                print(df.head())
                
                # Save to CSV
                filename = f"seed_dealers_{state_name}_{district_name}_{sample_market}.csv"
                df.to_csv(filename, index=False)
                print(f"\nData saved to: {filename}")
            
        except Exception as e:
            print(f"Failed to get market data: {e}")

=== PART 1: Getting all markets ===
Opening website...
Checking for initial popup...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Selecting state: Uttar Pradesh
Selecting district: Lucknow
Getting all available markets...
Found 8 markets: ['Bakshi-Ka-Talab', 'Chinhat', 'Gosaiganj', 'Kakori', 'Mal', 'Malihabad', 'Mohanlalganj', 'Sarojaninagar']

Available markets in Lucknow, Uttar Pradesh:
1. Bakshi-Ka-Talab
2. Chinhat
3. Gosaiganj
4. Kakori
5. Mal
6. Malihabad
7. Mohanlalganj
8. Sarojaninagar

=== PART 2: Getting data for specific market ===
Getting data for market: Bakshi-Ka-Talab
Opening website...
Checking for initial popup...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Selecting state: Uttar Pradesh
Selecting district: Lucknow
Selecting market: Bakshi-Ka-Talab
Clicking GO button...
GO button - Enabled: True, Visible: True
GO button clicked successfully
Checking for popup after GO click...
Found popup

In [38]:
# Get all markets first
markets = asyncio.run(get_markets("Uttar Pradesh", "Lucknow"))
print(markets)  # ['Bakshi-Ka-Talab', 'Chinhat', 'Gomti Nagar', ...]

# Then get data for a specific market
df = asyncio.run(scrape_market_data("Uttar Pradesh", "Lucknow", "Malihabad"))

Opening website...
Checking for initial popup...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Selecting state: Uttar Pradesh
Selecting district: Lucknow
Getting all available markets...
Found 8 markets: ['Bakshi-Ka-Talab', 'Chinhat', 'Gosaiganj', 'Kakori', 'Mal', 'Malihabad', 'Mohanlalganj', 'Sarojaninagar']
['Bakshi-Ka-Talab', 'Chinhat', 'Gosaiganj', 'Kakori', 'Mal', 'Malihabad', 'Mohanlalganj', 'Sarojaninagar']
Opening website...
Checking for initial popup...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Selecting state: Uttar Pradesh
Selecting district: Lucknow
Selecting market: Malihabad
Clicking GO button...
GO button - Enabled: True, Visible: True
GO button clicked successfully
Checking for popup after GO click...
Found popup with selector: #onloadModal
Clicked close button: #onloadModal .btn-close
Extracting table data for market: Malihabad
Extracted 12 rows for Malihabad
DataFrame created with colu

In [39]:
df

,Serial No,Type,District,Area,Dealer name,Mobile No,Address,NaPanta Mobile App
0,1,Seed Dealers,Lucknow,Malihabad,ANUJ KEETNASHAK KENDRA,None,THANA MAL,For Free Alerts
1,2,Seed Dealers,Lucknow,Malihabad,GUPTA BEEJ BHANDAR,None,KAKORI,For Free Alerts
2,3,Seed Dealers,Lucknow,Malihabad,KAMLASHANKER SHUKLA BEEJ BHANDAR,None,GHDNO MAL,For Free Alerts
3,4,Seed Dealers,Lucknow,Malihabad,KUMAR BEEJ BHANDAR,None,KAKORI,For Free Alerts
4,5,Seed Dealers,Lucknow,Malihabad,LALTA PRSAD VASH & SUNS,None,KAKORI MODH,For Free Alerts
5,6,Seed Dealers,Lucknow,Malihabad,Ms. P.C.F. KENDRA,None,DULARMAU,For Free Alerts
6,7,Seed Dealers,Lucknow,Malihabad,Ms. P.C.F. KENDRA,None,MALIHABAD,For Free Alerts
7,8,Seed Dealers,Lucknow,Malihabad,P.C.F,None,DULAR MAU,For Free Alerts
8,9,Seed Dealers,Lucknow,Malihabad,PHSAL SURKHA KANDRA,None,KHUSHAL PURVA,For Free Alerts
9,10,Seed Dealers,Lucknow,Malihabad,SAHU BEEJ BHANDAR,None,EBRAHIM GANJ,For Free Alerts
